In [1]:
import os

from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, DuckDuckGoSearchRun
from langchain.agents import Tool, load_agent
import mercantile

In [2]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [3]:
llm = ChatOpenAI(
    temperature=0,
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-3.5-turbo"
)

In [59]:
class MercantileTool(BaseTool):
    name="mercantile"
    description="Use this tool to get the xyz tiles given a lng,lat coordinate. \
    To use this tool you need to provide lng,lat,zoom level separated by comma. \
    Eg: `-105.24, 22.50, 5` is the input to get a tile for this (lng=-105.24, lat=22.50) at zoom level 5"

    def _run(self, query):
        lng,lat,zoom = map(float, query.split(","))
        return mercantile.tile(lng, lat, zoom)

    def _arun(self, query):
        raise NotImplementedError("Mercantile tool doesn't have an async implementation.")

In [73]:
duckduckgo_tool = Tool(
    name="DuckDuckGo",
    description="Use this tool to answer questions about current events. \
    Please ask targeted questions.",
    func=DuckDuckGoSearchRun().run
)

mercantile_tool = MercantileTool()

In [77]:
myagent = initialize_agent(
    agent="zero-shot-react-description",
    tools=[duckduckgo_tool, mercantile_tool],
    llm=llm,
    verbose=True,
    max_iterations=5,
    early_stopping_method="generate"
)

In [78]:
myagent("What is the xyz tile for London?")



> Entering new AgentExecutor chain...
I need to use mercantile to get the xyz tile for London, but first I need to find the longitude and latitude coordinates for London.
Action: DuckDuckGo
Action Input: "longitude and latitude coordinates for London"
Observation: latitude and longitude, coordinate system by means of which the position or location of any place on Earth's surface can be determined and described. Latitude is a measurement on a globe or map of location north or south of the Equator. ... London, England, the specially designated imaginary north-south line that passes through both ... The following table lists the latitude and longitude coordinates of major world cities in alphabetical order by city name. The time corresponding to 12:00 noon, Eastern Standard Time, is also included for each city. ... Latitude Longitude ... London, England: 51: 32 N: 0: 5 W: 5:00 p.m. Lyons, France: 45: 45 N: 4: 50 E: 6:00 p.m. Madrid, Spain ... Decimal degrees = Degrees + Minutes/60 + Sec

{'input': 'What is the xyz tile for London?',
 'output': 'Tile(x=512, y=340, z=10.0)'}